In [1]:
import os
import glob
import cv2
import pickle
import deeplabcut

import numpy as np

# Set up the 3D DeepLabCut project

Change the base path for each different 3D project

In [ ]:
base_path = r'D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d'

In [2]:
config_path_3d = os.path.join(base_path, 'config.yaml')

if os.path.exists(config_path_3d):
    print('Found existing 3d project')
else:
    print('Create new 3d project!')
    deeplabcut.create_new_project_3d('UpAndDown','Ming', 2)

Found existing 3d project


Make sure you edit the 3D config.yaml file. The cameras should be named 'rear' and 'lateral', and the config file paths for the 2D tracking should be updated correctly. Also update the skeleton to match the 2D tracking.

# Do the calibration

See https://github.com/DeepLabCut/DeepLabCut/blob/master/docs/Overviewof3D.md for an overview

Most of the code below is modified from the DLC repository here: https://github.com/DeepLabCut/DeepLabCut/blob/master/deeplabcut/pose_estimation_3d/camera_calibration.py

Number of internal corners on our checkboard. (e.g., 8x8 squares has 7x7 internal corners)

In [28]:
cbrow = 7
cbcol = 7

In [29]:
cam_names = ['rear', 'lateral']

In [30]:
framedir = os.path.join(base_path, 'calibration_images')
cornerdir = os.path.join(base_path, 'corners')

In [31]:
filenames = []
for cam1 in cam_names:
    fn1 = glob.glob(os.path.join(framedir, cam1 + '*.jpg'))
    fn1.sort()
    filenames.append(fn1)

filenames = np.array(filenames)

print(filenames[:,:3])

[['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\calibration_images\\rear-006.jpg'
  'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\calibration_images\\rear-007.jpg'
  'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\calibration_images\\rear-008.jpg']
 ['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\calibration_images\\lateral-006.jpg'
  'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\calibration_images\\lateral-007.jpg'
  'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\calibration_images\\lateral-008.jpg']]


In [32]:
rotategrid = [False, True]
mirror = [False, False]

In [33]:
# Termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [34]:
# Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((cbrow * cbcol, 3), np.float32)
objp[:, :2] = np.mgrid[0:cbcol, 0:cbrow].T.reshape(-1, 2)

In [35]:
img_shape = {}
objpoints = {}  # 3d point in real world space
imgpoints = {}  # 2d points in image plane.
dist_pickle = {}
stereo_params = {}
for cam in cam_names:
    objpoints.setdefault(cam, [])
    imgpoints.setdefault(cam, [])
    dist_pickle.setdefault(cam, [])

for cam, camfiles1, rot1, mirror1 in zip(cam_names, filenames, rotategrid, mirror):
    for fn1 in camfiles1:
        img = cv2.imread(fn1)
        if mirror1:
            img = cv2.flip(img, 1)  # flip horizontally
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
        pn, fn = os.path.split(fn1)
        fn, ext = os.path.splitext(fn)

        print('{}'.format(fn))
        ret, corners = cv2.findChessboardCorners(gray, (cbrow,cbcol), cv2.CALIB_CB_ADAPTIVE_THRESH + \
            cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)

        if ret:
            print('Found!')
            if rot1:
                corners = corners.reshape((cbrow,cbcol, -1))
                corners = corners.transpose((1, 0, 2))
                corners = corners.reshape((cbrow*cbcol, 1, -1))
            
            img_shape[cam] = gray.shape[::-1]
            objpoints[cam].append(objp)
            corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints[cam].append(corners)            
        else:
            print('Not found')

        img = cv2.drawChessboardCorners(img, (cbrow,cbcol), corners, ret)

        cv2.imwrite(os.path.join(cornerdir, fn + '_corner.jpg'), img)


rear-006
Found!
rear-007
Found!
rear-008
Found!
rear-009
Found!
rear-010
Found!
rear-011
Found!
rear-012
Found!
rear-013
Found!
rear-014
Found!
rear-016
Found!
rear-017
Found!
rear-018
Found!
rear-019
Found!
rear-020
Found!
rear-021
Found!
rear-022
Found!
rear-023
Found!
lateral-006
Found!
lateral-007
Found!
lateral-008
Found!
lateral-009
Found!
lateral-010
Found!
lateral-011
Found!
lateral-012
Found!
lateral-013
Found!
lateral-014
Found!
lateral-016
Found!
lateral-017
Found!
lateral-018
Found!
lateral-019
Found!
lateral-020
Found!
lateral-021
Found!
lateral-022
Found!
lateral-023
Found!


In [36]:
path_camera_matrix = os.path.join(base_path,'camera_matrix')

In [37]:
alpha = 0.8

In [38]:
for cam in cam_names:
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
        objpoints[cam], imgpoints[cam], img_shape[cam], None, None
    )

    # Save the camera calibration result for later use (we won't use rvecs / tvecs)
    dist_pickle[cam] = {
        "mtx": mtx,
        "dist": dist,
        "objpoints": objpoints[cam],
        "imgpoints": imgpoints[cam],
    }
    pickle.dump(
        dist_pickle,
        open(
            os.path.join(path_camera_matrix, cam + "_intrinsic_params.pickle"),
            "wb",
        ),
    )
    print(
        "Saving intrinsic camera calibration matrices for %s as a pickle file in %s"
        % (cam, os.path.join(path_camera_matrix))
    )

    # Compute mean re-projection errors for individual cameras
    mean_error = 0
    for i in range(len(objpoints[cam])):
        imgpoints_proj, _ = cv2.projectPoints(
            objpoints[cam][i], rvecs[i], tvecs[i], mtx, dist
        )
        error = cv2.norm(imgpoints[cam][i], imgpoints_proj, cv2.NORM_L2) / len(
            imgpoints_proj
        )
        mean_error += error
    print(
        "Mean re-projection error for %s images: %.3f pixels "
        % (cam, mean_error / len(objpoints[cam]))
    )


Saving intrinsic camera calibration matrices for rear as a pickle file in D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\camera_matrix
Mean re-projection error for rear images: 0.189 pixels 
Saving intrinsic camera calibration matrices for lateral as a pickle file in D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\camera_matrix
Mean re-projection error for lateral images: 0.181 pixels 


In [39]:
h, w = img.shape[:2]

In [40]:
# Compute stereo calibration for each pair of cameras
camera_pair = [[cam_names[0], cam_names[1]]]
for pair in camera_pair:
    print("Computing stereo calibration for " % pair)
    (
        retval,
        cameraMatrix1,
        distCoeffs1,
        cameraMatrix2,
        distCoeffs2,
        R,
        T,
        E,
        F,
    ) = cv2.stereoCalibrate(
        objpoints[pair[0]],
        imgpoints[pair[0]],
        imgpoints[pair[1]],
        dist_pickle[pair[0]]["mtx"],
        dist_pickle[pair[0]]["dist"],
        dist_pickle[pair[1]]["mtx"],
        dist_pickle[pair[1]]["dist"],
        (h, w),
        flags=cv2.CALIB_FIX_INTRINSIC,
    )

    # Stereo Rectification
    rectify_scale = alpha  # Free scaling parameter check this https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#fisheye-stereorectify
    R1, R2, P1, P2, Q, roi1, roi2 = cv2.stereoRectify(
        cameraMatrix1,
        distCoeffs1,
        cameraMatrix2,
        distCoeffs2,
        (h, w),
        R,
        T,
        alpha=rectify_scale,
    )

    stereo_params[pair[0] + "-" + pair[1]] = {
        "cameraMatrix1": cameraMatrix1,
        "cameraMatrix2": cameraMatrix2,
        "distCoeffs1": distCoeffs1,
        "distCoeffs2": distCoeffs2,
        "R": R,
        "T": T,
        "E": E,
        "F": F,
        "R1": R1,
        "R2": R2,
        "P1": P1,
        "P2": P2,
        "roi1": roi1,
        "roi2": roi2,
        "Q": Q,
        "image_shape": [img_shape[pair[0]], img_shape[pair[1]]],
    }

print(
    "Saving the stereo parameters for every pair of cameras as a pickle file in %s"
    % str(os.path.join(path_camera_matrix))
)

deeplabcut.auxiliaryfunctions.write_pickle(
    os.path.join(path_camera_matrix, "stereo_params.pickle"), stereo_params
)
print(
    "Camera calibration done! Use the function ``check_undistortion`` to check the check the calibration"
)

Computing stereo calibration for 
Saving the stereo parameters for every pair of cameras as a pickle file in D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\camera_matrix
Camera calibration done! Use the function ``check_undistortion`` to check the check the calibration


In [41]:
cameraMatrix2

array([[5.08822638e+03, 0.00000000e+00, 1.26291971e+03],
       [0.00000000e+00, 5.37273202e+03, 5.51275139e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

## Check undistortion

This doesn't work, but the triangulation does. Just skip to the triangulation section

In [17]:
deeplabcut.check_undistortion(config_path_3d, cbrow=cbrow, cbcol=cbcol)

All images are undistorted and stored in D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\undistortion
Use the function ``triangulate`` to undistort the dataframes and compute the triangulation


In [83]:
camera_pair = [[cam_names[0], cam_names[1]]]

In [84]:
path_undistort = os.path.join(base_path, 'undistortion')

In [85]:
pairname = camera_pair[0][0] + "-" + camera_pair[0][1]
map2_x, map2_y = cv2.initUndistortRectifyMap(
    stereo_params[pairname]["cameraMatrix2"],
    stereo_params[pairname]["distCoeffs2"],
    stereo_params[pairname]["R2"],
    stereo_params[pairname]["P2"],
    (stereo_params[pairname]["image_shape"][1]),
    cv2.CV_16SC2,
)
 

In [87]:
map2_x

array([[[ 32767,  32767],
        [ 32767,  32767],
        [ 32767,  32767],
        ...,
        [-32768, -32768],
        [-32768, -32768],
        [-32768, -32768]],

       [[ 32767,  32767],
        [ 32767,  32767],
        [ 32767,  32767],
        ...,
        [-32768, -32768],
        [-32768, -32768],
        [-32768, -32768]],

       [[ 32767,  32767],
        [ 32767,  32767],
        [ 32767,  32767],
        ...,
        [-32768, -32768],
        [-32768, -32768],
        [-32768, -32768]],

       ...,

       [[ 32767, -32768],
        [ 32767, -32768],
        [ 32767, -32768],
        ...,
        [-32768, -32768],
        [-32768, -32768],
        [-32768, -32768]],

       [[ 32767, -32768],
        [ 32767, -32768],
        [ 32767, -32768],
        ...,
        [-32768, -32768],
        [-32768, -32768],
        [-32768, -32768]],

       [[ 32767, -32768],
        [ 32767, -32768],
        [ 32767, -32768],
        ...,
        [-32768, -32768],
        [-32768

In [35]:
for pair in camera_pair:
        map1_x, map1_y = cv2.initUndistortRectifyMap(
            stereo_params[pair[0] + "-" + pair[1]]["cameraMatrix1"],
            stereo_params[pair[0] + "-" + pair[1]]["distCoeffs1"],
            stereo_params[pair[0] + "-" + pair[1]]["R1"],
            stereo_params[pair[0] + "-" + pair[1]]["P1"],
            (stereo_params[pair[0] + "-" + pair[1]]["image_shape"][0]),
            cv2.CV_16SC2,
        )
        map2_x, map2_y = cv2.initUndistortRectifyMap(
            stereo_params[pair[0] + "-" + pair[1]]["cameraMatrix2"],
            stereo_params[pair[0] + "-" + pair[1]]["distCoeffs2"],
            stereo_params[pair[0] + "-" + pair[1]]["R2"],
            stereo_params[pair[0] + "-" + pair[1]]["P2"],
            (stereo_params[pair[0] + "-" + pair[1]]["image_shape"][1]),
            cv2.CV_16SC2,
        )
        cam1_undistort = []
        cam2_undistort = []

        for camnum, (cam, camfiles1, rot1) in enumerate(zip(cam_names, filenames, rotategrid)):
            for fname in camfiles1:
                _, filename = os.path.split(fname)
                img1 = cv2.imread(fname)
                gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
                h, w = img1.shape[:2]
                _, corners1 = cv2.findChessboardCorners(gray1, (cbcol, cbrow),  cv2.CALIB_CB_ADAPTIVE_THRESH + \
            cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
                
                if rot1:
                    corners1 = corners1.reshape((cbrow,cbcol, -1))
                    corners1 = corners1.transpose((1, 0, 2))
                    corners1 = corners1.reshape((cbrow*cbcol, 1, -1))

                corners_origin1 = cv2.cornerSubPix(
                    gray1, corners1, (11, 11), (-1, -1), criteria
                )
                

                # Remapping dataFrame_camera1_undistort
                im_remapped1 = cv2.remap(gray1, map1_x, map1_y, cv2.INTER_LANCZOS4)
                imgpoints_proj_undistort = cv2.undistortPoints(
                    src=corners_origin1,
                    cameraMatrix=stereo_params[pair[0] + "-" + pair[1]][
                        "cameraMatrix{}".format(camnum+1)
                    ],
                    distCoeffs=stereo_params[pair[0] + "-" + pair[1]]["distCoeffs{}".format(camnum+1)],
                    P=stereo_params[pair[0] + "-" + pair[1]]["P{}".format(camnum+1)],
                    R=stereo_params[pair[0] + "-" + pair[1]]["R{}".format(camnum+1)],
                )
                cam1_undistort.append(imgpoints_proj_undistort)
                cv2.imwrite(
                    os.path.join(str(path_undistort), filename + "_undistort.jpg"),
                    im_remapped1,
                )
                imgpoints_proj_undistort = []

KeyboardInterrupt: 

In [ ]:

        cam1_undistort = np.array(cam1_undistort)
        cam2_undistort = np.array(cam2_undistort)
        print("All images are undistorted and stored in %s" % str(path_undistort))
        print(
            "Use the function ``triangulate`` to undistort the dataframes and compute the triangulation"
        )

        if plot == True:
            f1, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
            f1.suptitle(
                str("Original Image: Views from " + pair[0] + " and " + pair[1]),
                fontsize=25,
            )

            # Display images in RGB
            ax1.imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
            ax2.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))

            norm = mcolors.Normalize(vmin=0.0, vmax=cam1_undistort.shape[1])
            plt.savefig(os.path.join(str(path_undistort), "Original_Image.png"))

            # Plot the undistorted corner points
            f2, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
            f2.suptitle(
                "Undistorted corner points on camera-1 and camera-2", fontsize=25
            )
            ax1.imshow(cv2.cvtColor(im_remapped1, cv2.COLOR_BGR2RGB))
            ax2.imshow(cv2.cvtColor(im_remapped2, cv2.COLOR_BGR2RGB))
            for i in range(0, cam1_undistort.shape[1]):
                ax1.scatter(
                    [cam1_undistort[-1][i, 0, 0]],
                    [cam1_undistort[-1][i, 0, 1]],
                    marker=markerType,
                    s=markerSize,
                    color=markerColor,
                    alpha=alphaValue,
                )
                ax2.scatter(
                    [cam2_undistort[-1][i, 0, 0]],
                    [cam2_undistort[-1][i, 0, 1]],
                    marker=markerType,
                    s=markerSize,
                    color=markerColor,
                    alpha=alphaValue,
                )
            plt.savefig(os.path.join(str(path_undistort), "undistorted_points.png"))

            # Triangulate
            triangulate = auxiliaryfunctions_3d.compute_triangulation_calibration_images(
                stereo_params[pair[0] + "-" + pair[1]],
                cam1_undistort,
                cam2_undistort,
                path_undistort,
                cfg_3d,
                plot=True,
            )
            auxiliaryfunctions.write_pickle("triangulate.pickle", triangulate)

# Triangulate

In [3]:
deeplabcut.triangulate(config_path_3d, r'D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos',
                    videotype='.mp4', save_as_csv=True)

List of pairs: [['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211013_ms03_trial06_rear.mp4', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211013_ms03_trial06_lateral.mp4'], ['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial01_rear.mp4', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial01_lateral.mp4'], ['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial02_rear.mp4', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial02_lateral.mp4']]
Analyzing video D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos\20211013_ms03_trial06_rear.mp4 using config_file_rear
Already analyzed...Checking the meta data for any change in the camera matrices and/or scorer names 20211013_ms03_trial06_rear
This file is already analyzed!
Analyzing video D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos\20211013_ms03_trial06_lateral.mp4 using config_file_lateral
Already analyzed...Checking the meta data for

In [43]:
deeplabcut.create_labeled_video_3d(config_path_3d, [r'D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos'],
                 videotype='.mp4',
                 start=100, end=200)

Analyzing all the videos in the directory
[['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211013_ms03_trial06_DLC_3D.h5', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211013_ms03_trial06_rear.mp4', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211013_ms03_trial06_lateral.mp4'], ['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial01_DLC_3D.h5', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial01_rear.mp4', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial01_lateral.mp4'], ['D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial02_DLC_3D.h5', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial02_rear.mp4', 'D:\\UpAndDown\\UpAndDown-Ming-2022-02-22-3d\\videos\\20211028_ms03_trial02_lateral.mp4']]
Creating 3D video from 20211013_ms03_trial06_rear.mp4 and 20211013_ms03_trial06_lateral.mp4 using 20211013_ms03_trial06_DLC_3D.h5
Looking for filtered predi

100%|██████████| 100/100 [03:21<00:00,  2.02s/it]


Creating 3D video from 20211028_ms03_trial01_rear.mp4 and 20211028_ms03_trial01_lateral.mp4 using 20211028_ms03_trial01_DLC_3D.h5
Looking for filtered predictions...
Found the following filtered data:  D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos\*20211028_ms03_trial01_rearDLC_resnet50_UpAndDownRearNov27shuffle1_1030000*filtered.h5 D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos\*20211028_ms03_trial01_lateralDLC_resnet50_UpAndDownLateral2Feb7shuffle1_1030000*filtered.h5


100%|██████████| 100/100 [03:16<00:00,  1.96s/it]


Creating 3D video from 20211028_ms03_trial02_rear.mp4 and 20211028_ms03_trial02_lateral.mp4 using 20211028_ms03_trial02_DLC_3D.h5
Looking for filtered predictions...
Found the following filtered data:  D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos\*20211028_ms03_trial02_rearDLC_resnet50_UpAndDownRearNov27shuffle1_1030000*filtered.h5 D:\UpAndDown\UpAndDown-Ming-2022-02-22-3d\videos\*20211028_ms03_trial02_lateralDLC_resnet50_UpAndDownLateral2Feb7shuffle1_1030000*filtered.h5


100%|██████████| 100/100 [03:17<00:00,  1.97s/it]
